# Integer indexing

Indexing with an integer selects a single position within the corresponding dimension:

In [3]:
import tensorstore as ts
a = ts.array([[0, 1, 2], [3, 4, 5]], dtype=ts.int32)
a[0]

TensorStore({
  'array': [0, 1, 2],
  'context': {'data_copy_concurrency': {}},
  'driver': 'array',
  'dtype': 'int32',
  'transform': {'input_exclusive_max': [3], 'input_inclusive_min': [0]},
})

In [2]:
a[1,2]
## 5 being the value at the 1st row and 2nd column of the array

TensorStore({
  'array': 5,
  'context': {'data_copy_concurrency': {}},
  'driver': 'array',
  'dtype': 'int32',
  'transform': {'input_rank': 0},
})

Because TensorStore supports index domains defined over negative indices, negative values have no special meaning; they simply refer to negative positions:

In [24]:
a = await ts.open({
    "dtype": "int32",
    "driver": "array",
    "array": [1, 2, 3],
    "transform": {
        "input_shape": [3],
        "input_inclusive_min": [-10], # -10, -9, -8 are valid 1,2,3 indices
        "output": [{
            "input_dimension": 0,
            "offset": 10
        }],
    },
})
print(a[-10])
print(a[-9])
print(a[-8])

TensorStore({
  'array': 1,
  'context': {'data_copy_concurrency': {}},
  'driver': 'array',
  'dtype': 'int32',
  'transform': {'input_rank': 0},
})
TensorStore({
  'array': 2,
  'context': {'data_copy_concurrency': {}},
  'driver': 'array',
  'dtype': 'int32',
  'transform': {'input_rank': 0},
})
TensorStore({
  'array': 3,
  'context': {'data_copy_concurrency': {}},
  'driver': 'array',
  'dtype': 'int32',
  'transform': {'input_rank': 0},
})


Specifying an index outside the implicit bounds of a dimension is permitted:

In [25]:
a = ts.IndexTransform(input_shape=[4], implicit_lower_bounds=[True])
a[-1]

Rank 0 -> 1 index space transform:
  Input domain:
  Output index maps:
    out[0] = -1

In [30]:
a[3]
## a[4] results into an error
## Error : Checking bounds of constant output index map for dimension 0: Index 4 is outside valid range (-inf, 4)

Rank 0 -> 1 index space transform:
  Input domain:
  Output index maps:
    out[0] = 3

## Interval indexing

Indexing with a slice object start:stop:step selects an interval or strided interval within the corresponding dimension:

In [37]:
a = ts.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=ts.int32)
print(a[1:5])
print(a[1:5][2])

TensorStore({
  'array': [1, 2, 3, 4],
  'context': {'data_copy_concurrency': {}},
  'driver': 'array',
  'dtype': 'int32',
  'transform': {
    'input_exclusive_max': [5],
    'input_inclusive_min': [1],
    'output': [{'input_dimension': 0, 'offset': -1}],
  },
})
TensorStore({
  'array': 2,
  'context': {'data_copy_concurrency': {}},
  'driver': 'array',
  'dtype': 'int32',
  'transform': {'input_rank': 0},
})


In [38]:
## steps
print(a[3:8:2])
print(a[7:3:-2])

TensorStore({
  'array': [3, 5, 7],
  'context': {'data_copy_concurrency': {}},
  'driver': 'array',
  'dtype': 'int32',
  'transform': {
    'input_exclusive_max': [4],
    'input_inclusive_min': [1],
    'output': [{'input_dimension': 0, 'offset': -1}],
  },
})
TensorStore({
  'array': [7, 5],
  'context': {'data_copy_concurrency': {}},
  'driver': 'array',
  'dtype': 'int32',
  'transform': {
    'input_exclusive_max': [-1],
    'input_inclusive_min': [-3],
    'output': [{'input_dimension': 0, 'offset': 3}],
  },
})


Specifying an interval outside the implicit bounds of a dimension is permitted:

In [39]:
a = ts.IndexTransform(input_shape=[4], implicit_lower_bounds=[True])
a[-1:2]

Rank 1 -> 1 index space transform:
  Input domain:
    0: [-1, 2)
  Output index maps:
    out[0] = 0 + 1 * in[0]

Any of the start, stop, or stop values may be specified as a sequence of integer or None values (e.g. a list, tuple or 1-d numpy.ndarray), rather than a single integer:

In [47]:
a = ts.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]], dtype=ts.int32)
print(a[(1,1): (3,4)])

# Equivalent to specifying a sequence of slice objects:

print(a[1:3, 1:4])

TensorStore({
  'array': [[6, 7, 8], [10, 11, 12]],
  'context': {'data_copy_concurrency': {}},
  'driver': 'array',
  'dtype': 'int32',
  'transform': {
    'input_exclusive_max': [3, 4],
    'input_inclusive_min': [1, 1],
    'output': [
      {'input_dimension': 0, 'offset': -1},
      {'input_dimension': 1, 'offset': -1},
    ],
  },
})
TensorStore({
  'array': [[6, 7, 8], [10, 11, 12]],
  'context': {'data_copy_concurrency': {}},
  'driver': 'array',
  'dtype': 'int32',
  'transform': {
    'input_exclusive_max': [3, 4],
    'input_inclusive_min': [1, 1],
    'output': [
      {'input_dimension': 0, 'offset': -1},
      {'input_dimension': 1, 'offset': -1},
    ],
  },
})
TensorStore({
  'array': [[6, 7, 8], [10, 11, 12]],
  'context': {'data_copy_concurrency': {}},
  'driver': 'array',
  'dtype': 'int32',
  'transform': {
    'input_exclusive_max': [3, 4],
    'input_inclusive_min': [1, 1],
    'output': [
      {'input_dimension': 0, 'offset': -1},
      {'input_dimension': 1, '

It is an error to specify a slice with sequences of unequal lengths, but a sequence may be combined with a scalar value:

In [48]:
print(a[1:(3,4)])

TensorStore({
  'array': [[6, 7, 8], [10, 11, 12]],
  'context': {'data_copy_concurrency': {}},
  'driver': 'array',
  'dtype': 'int32',
  'transform': {
    'input_exclusive_max': [3, 4],
    'input_inclusive_min': [1, 1],
    'output': [
      {'input_dimension': 0, 'offset': -1},
      {'input_dimension': 1, 'offset': -1},
    ],
  },
})


## Adding singleton dimensions

Specifying a value of tensorstore.newaxis (equal to None) adds a new dummy/singleton dimension with implicit bounds [0,1):

In [49]:
a = ts.IndexTransform(input_rank=2)
a[ts.newaxis]

Rank 3 -> 2 index space transform:
  Input domain:
    0: [0*, 1*)
    1: (-inf*, +inf*)
    2: (-inf*, +inf*)
  Output index maps:
    out[0] = 0 + 1 * in[1]
    out[1] = 0 + 1 * in[2]

This indexing term adds a single dimension after any dimensions added by prior indexing operations:

In [51]:
a[:, ts.newaxis, ts.newaxis]

Rank 4 -> 2 index space transform:
  Input domain:
    0: (-inf*, +inf*)
    1: [0*, 1*)
    2: [0*, 1*)
    3: (-inf*, +inf*)
  Output index maps:
    out[0] = 0 + 1 * in[0]
    out[1] = 0 + 1 * in[3]